In [3]:
## Load modules
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import os
cwd = os.getcwd() # current working directory

In [4]:
import sys
sys.path.append(os.path.abspath(cwd))

In [3]:
## Load data
#data_dir = "cycle_10bins_conditions_none"#"2sec_data_avg" # name of the data folder

# Loading the train, dev, test data from specified folder
#os.chdir(cwd+"/"+data_dir+"/train")
#x_train = genfromtxt("x_train.csv", delimiter=',')
#y_train = genfromtxt("y_train.csv", delimiter=',')

In [5]:
# data (num_samples, features)
data = genfromtxt("Firewheel_FWL1_mast_file_tsec_72-months.csv", delimiter=',')

In [6]:
# take input data
# add a for loop for each data file do this with batches
# break into batches
batch_size = 512
num_samples, features = data.shape
num_batches = int(num_samples/batch_size)
batch_ind = # (file_count, batch_start_idx)
# randomly assign batches to train/dev/test
# pull out y values
# normalize to training batches
# reshape_sequences

print(x_all.shape)

x_train = data[10:1010,10:20] # (1000,10)
data_size, features = x_train.shape

# need to get the data from the files, split by sequence length and 

(315511, 31)


In [12]:
def normalize_data(data):
    mu = np.mean(data,axis=0) # compute the mean along axis = 0 (num_samples for raw data)
    cov = np.std(data,axis=0) # using std instead of variance seems to be best
    return mu, cov # returning the normalizations for the data

x_mu, x_cov = normalize_data(x_train) # x_train is (features x num_samples)
#y_mu, y_cov = normalize_data(y_train)

X_train = ((x_train - x_mu)/x_cov) # still in (data examples, input_size)

# have (# data examples, inpute_size)
# want (# data examples - num_steps - 1, num_steps, input_size)
# given the index for prediction data (y_ind), creates prediction vector Y
def reshape_sequences(data, sequence_length, y_ind):
    data_size, features = data.shape
    out_data = np.zeros((data_size-sequence_length-1, sequence_length, features))
    out_y = np.zeros(data_size-sequence_length-1)
    for i in range(out_data.shape[0]):
        out_data[i,:,:] = data[i:sequence_length+i,:]
        out_y[i] = data[sequence_length+i+1,y_ind]
    return out_data, out_y

sequence_length = 20
y_ind = 3
X_train, Y_train = reshape_sequences(x_train, sequence_length, y_ind)

print(Y_train.shape)
print(X_train.shape)
#print(np.mean(X_train[:,1]))
#X_dev = ((x_dev - x_mu)/x_cov).T # Use same distrib for others --> don't use "future" data
#X_test = ((x_test - x_mu)/x_cov).T

(979,)
(979, 20, 10)


In [6]:
def initialize_parameters(num_hid_layers, size_hid_layers, n_x, output_size):
    parameters = {}
    total_layers = num_hid_layers+1
    
    for l in range(1,total_layers+1):
        if l == 1:
            a = size_hid_layers
            b = n_x
        elif l == total_layers:
            a = output_size
            b = size_hid_layers
        else:
            a = size_hid_layers
            b = size_hid_layers
            
        parameters['w' + str(l)] = tf.get_variable('w'+str(l), [a, b], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        parameters['b' + str(l)] = tf.get_variable('b'+str(l), [a,1], initializer = tf.zeros_initializer())    
    return parameters

In [7]:
def forward_prop(X,parameters):
    total_layers = len(parameters)//2
    layer_outputs = {}
    layer_outputs['A0'] = X
    
    for l in range(1,total_layers+1):
        layer_outputs['Z' + str(l)] = tf.matmul(parameters['w' + str(l)],layer_outputs['A' + str(l-1)])+parameters['b' + str(l)]
        layer_outputs['A' + str(l)] = tf.nn.relu(layer_outputs['Z' + str(l)])
        #layer_outputs['A' + str(l)] = tf.nn.dropout(layer_outputs['A' + str(l)],keep_prob)
    
    return layer_outputs['Z' + str(total_layers)]

In [39]:
def model(X_train, Y_train, learning_rate, num_epochs, num_hid_layers, size_hid_layers, minibatch_size, print_interval=10):

    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    (m, seq_length, n_x) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    losses = []                                        # To keep track of the cost
    
    O = 1 # output size?
    w1_shape = [O, size_hid_layers]
    b1_shape = [O,1]
    
    w1 = tf.get_variable("w1", w1_shape, initializer=tf.contrib.layers.xavier_initializer(seed=1))
    b1 = tf.get_variable("b1", b1_shape, initializer = tf.zeros_initializer())
    
    X = tf.placeholder(tf.float32,[None, seq_length, n_x]) # inputs to LSTM are (# data examples, num_steps, input_size)
    Y = tf.placeholder(tf.float32,[None])

    # create 2 LSTMCells
    rnn_layers = [tf.nn.rnn_cell.LSTMCell(size_hid_layers) for size_hid_layers in [size_hid_layers, size_hid_layers]]

    # create a RNN cell composed sequentially of a number of RNNCells
    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

    outputs, state = tf.nn.dynamic_rnn(cell=multi_rnn_cell, inputs=X, dtype=tf.float32)
    #print(outputs.shape)
    #print(outputs[:,1,:].shape)
    # WANT 1 OUTPUT value so outputs is ideally hidden_layer x # data samples, but currently its # data samples, seq, features
    # the 20 outputs are for each sequence step, want the last one I believe
    Z1 = tf.matmul(w1,tf.transpose(outputs[:,-1,:]))+b1 # fully connected layer
    out = Z1
    
    #parameters = initialize_parameters(num_hid_layers, size_hid_layers, n_x, O)
    #out = forward_prop(X, parameters)
    
    loss = tf.reduce_mean(tf.squared_difference(out, Y)) # L2 loss --> not good for our problem
    #loss = tf.reduce_mean(tf.losses.absolute_difference(Y,tf.squeeze(out))) # L1 loss
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss) # Optimizer, change the learning rate here

    init = tf.global_variables_initializer() # When init is run later (session.run(init)),
    with tf.Session() as sess: # starting tf session --> all computation on tf graph in this with struct
        sess.run(init)
        for epoch in range(num_epochs+1):
            _, loss_val = sess.run([optimizer, loss], feed_dict={X: X_train, Y: Y_train})
            losses.append(loss_val)
            if epoch % (num_epochs/print_interval) == 0:
                print("Loss: ",loss_val)
            

In [40]:
learning_rate = 0.008#0.0005
num_epochs = 1000 # total number of epochs to iterate through
print_interval = 10 # number of prints per total run
minibatch_size = 10
num_hid_layers = 3
size_hid_layers = 128

model(X_train, Y_train, learning_rate, num_epochs, num_hid_layers, size_hid_layers, minibatch_size, print_interval)

Loss:  1.9385155
Loss:  0.00017618242
Loss:  9.377167e-05
Loss:  6.67244e-05
Loss:  5.3395765e-05
Loss:  4.562433e-05
Loss:  4.078155e-05
Loss:  3.6908623e-05
Loss:  3.4504985e-05
Loss:  3.2830936e-05
Loss:  3.163642e-05
